In [ ]:
import os
import random
import time

import timm
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import pandas as pd
from collections import Counter
import numpy as np
import torch.nn as nn
from sklearn.model_selection import KFold
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from torch.optim import AdamW, RMSprop, SGD, Adam, Adamax, Adadelta, Adagrad
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, StepLR

In [ ]:
# 앙상블할 csv 파일들
enssemble_file1 = pd.read_csv('/home/data/pred/kfold_pred/vit_base_patch16_epoch5_pth1.csv')
enssemble_file2 = pd.read_csv('/home/data/pred/kfold_pred/vit_base_patch16_epoch5_pth2.csv')
enssemble_file3 = pd.read_csv('/home/data/pred/kfold_pred/vit_base_patch16_epoch5_pth3.csv')

# 앙상블할 csv 파일들의 target값
enssemble_file1_target = enssemble_file1["target"]
enssemble_file2_target = enssemble_file2["target"]
enssemble_file3_target = enssemble_file3["target"]

enssemble_target = []

for i in range(len(enssemble_file1_target)):
    votes = [
        enssemble_file1_target[i],
        enssemble_file2_target[i],
        enssemble_file3_target[i]
    ]
    
    most_common = Counter(votes).most_common(1)
    enssemble_target.append(most_common[0][0])
    print(f'{i}번째 target: {most_common}, {enssemble_target[i]}')

In [ ]:
# 결과 저장
result_df = pd.DataFrame({
    'ID': enssemble_file1['ID'],
    'target': enssemble_target
})
result_df.to_csv("/home/data/pred/enssemble_pred/mybest_hardvoting_vit_base_epoch5.csv", index=False)

In [ ]:
answer_df = pd.read_csv("/home/data/answer.csv")
assert (answer_df['ID'] == result_df['ID']).all()

f1 = f1_score(answer_df['target'], result_df['target'], average='macro')
print(f"Macro F1 Score: {f1:.4f}")